# Step-02: Master flat, X Y inclinations & spectral line

## Import dependencies

In [1]:
from __future__ import print_function
% gui tk
% matplotlib tk
from ReductionImports import *
setrcParams()
INITDIR = getFileFilesDir(2)
print(INITDIR)

E:/CaII Polarimeter v2


# Matser flat

## Inputs
> Single file of stack of flats $N \times Y \times W $ <br>
> Master dark file $Y \times W$ <br>
> Corrected fringe flat $Y \times W$

## Outputs
> Master flat file saved in Level-1 directory $Y \times W$ <br>
> Row and column shifts saved in Level-1 directory as text <br>
> Line profile saved in Level-1 directory as text <br>

## Process
> Stack of $N$ number of flat files, master dark are loaded <br>
> Flats are averaged then master dark is subtracted <br>
> X inclination is corrected by shifting the columns <br>
> Column shift values are obtained by tracing the slit profile <br>
> Y inclination is corrected by shifting the rows <br>
> Row shift values are obtained by tracing a fine spectral line <br>
> For this trace, flat is corrected by fringe flat to reduce the contrast of fringes due to _etaloning effect_ <br>
> After X, Y inclination corrections, image is aligned and medial spectral line is calculated <br>
> Half of rows from middle of top and bottom beams of the image with reduced fringe contrast are used for this <br>
> Each row in flat is divided with the line profile <br>
> Result is formatted and saved as _FLATMASTER_

## Load files

In [50]:
FLATFILENAME = getFileFilesDir(0, initialdir = INITDIR,
                               filetypes  = (('FITS file', '*FLAT*.FITS'),),
                               title = 'Select the stack of flat files...') # Select the file
HDU = pf.open(FLATFILENAME)
FLAT = HDU[0].data
HEAD = HDU[0].header
HDU.close()
print(FLATFILENAME, ' is loaded')

E:/CaII Polarimeter v2/20180428/Flats/090710_FLAT.fits  is loaded


In [ ]:
DARKFILENAME = getFileFilesDir(0, initialdir = INITDIR,
                               filetypes  = (('FITS file', '*DARK*MASTER*.FITS'),),
                               title = 'Select the master dark...') # Select the file
HDU = pf.open(DARKFILENAME)
DARKMASTER = float32(HDU[0].data)
DARKHEAD = HDU[0].header
HDU.close()
print(DARKFILENAME, ' is loaded')

In [ ]:
FRINGFILENAME = getFileFilesDir(0, initialdir = INITDIR,
                               filetypes  = (('FITS file', '*FRING*.FITS'),),
                               title = 'Select the corrected fringes file...') # Select the file
HDU = pf.open(FRINGFILENAME)
FRING = float32(HDU[0].data)
HDU.close()
print(FRINGFILENAME, ' is loaded')

## Dark subtraction

In [53]:
FLATMEAN = average(FLAT,axis=0)
FLATMEAN_DARKCORR = FLATMEAN - DARKMASTER
previewImage(FLATMEAN_DARKCORR)

## X inclinations

In [54]:
MAXSHIFT, EXTENT, UPSAMP = 8, 10, 10 
HPIX = FLATMEAN_DARKCORR.shape[1]
HORPIX, SHIFT_VER, SHIFTS_VER, SHIFTS_VER_FIT, WEIGHTS = arange(HPIX), zeros(HPIX), zeros(HPIX), zeros(HPIX), ones(HPIX)
DISP = copy(FLATMEAN_DARKCORR)
figure('Click on the slit profile to trace')
imshow(DISP, cmap='gray')
PT = map(int, ginput(1)[0])
close()
REFCOL, YBEG, YEND = PT[0], PT[1]-EXTENT/2, PT[1]+EXTENT/2
SLIT_REF = DISP[YBEG:YEND, REFCOL] # Reference slit profile
tempR = (SLIT_REF - SLIT_REF.mean())/SLIT_REF.std() # Normalize the reference slit profile
for j in HORPIX:
    tempV = DISP[YBEG-MAXSHIFT:YEND+MAXSHIFT,j]
    WEIGHTS[j] = sqrt(tempV.mean()**2)
    tempV = (tempV - tempV.mean())/tempV.std() # Normalize the selected slit profile
    # Correlate
    CORR = correlate(zoom(tempV, UPSAMP), zoom(tempR, UPSAMP), mode='valid')
    SHIFT_VER[j] = argmax(CORR) # Get the index with maximum correlation
SHIFT_VER = SHIFT_VER/UPSAMP - MAXSHIFT # Convert the index to pixel shift
XFIT_XINC = argwhere(abs(SHIFT_VER)<MAXSHIFT)
YFIT_XINC = SHIFT_VER[XFIT_XINC]
C = polyfit(XFIT_XINC.ravel(), YFIT_XINC.ravel(), 1, w=nan_to_num(WEIGHTS)[XFIT_XINC].ravel()) # Fit a line
# C = polyfit(XFIT_XINC.ravel(), YFIT_XINC.ravel(), 1) # Fit a line
SHIFT_VER_FIT = C[0]*HORPIX+C[1] # Use the equation to calculate shifts
SHIFT_VER_APPLY = -SHIFT_VER_FIT
previewPlot(XFIT_XINC, YFIT_XINC, 'k-', SHIFT_VER_FIT, 'k-')

In [55]:
FLATMEAN_XINCCORR = copy(FLATMEAN_DARKCORR) # Declare shift corrected array
for i in HORPIX: # Calculate the shift corrected array
    shift(FLATMEAN_DARKCORR[:, i], SHIFT_VER_APPLY[i], FLATMEAN_XINCCORR[:, i], mode='nearest')
previewImage(FLATMEAN_XINCCORR)

## Fringe corrections

In [ ]:
FRING[757:767,840:846] = FRING.mean() # Declare shift corrected array
FRING_XINCCORR = copy(FRING)
for i in HORPIX: # Calculate the shift corrected array
    shift(FRING[:, i], SHIFT_VER_APPLY[i], FRING_XINCCORR[:, i], mode='nearest')
FLATMEAN_XINCCORR2 = FLATMEAN_XINCCORR/FRING_XINCCORR
previewImage(FLATMEAN_XINCCORR2)

In [56]:
FLATMEAN_XINCCORR2 = copy(FLATMEAN_XINCCORR)
previewImage(FLATMEAN_XINCCORR2)

## Y-inclination, distortion

In [57]:
MAXSHIFT, EXTENT, UPSAMP = 10, 20, 10 
VPIX = FLATMEAN_DARKCORR.shape[0] # Total rows
VERPIX, SHIFT_HOR, SHIFTS_HOR, SHIFTS_HOR_FIT, WEIGHTS = arange(VPIX), zeros(VPIX), zeros(VPIX), zeros(VPIX), ones(VPIX)
DISP = copy(FLATMEAN_XINCCORR2)
figure('Click on the spectral line profile to trace')
imshow(DISP, cmap='gray')
PT = map(int, ginput(1)[0])
close()
REFROW, XBEG, XEND = PT[1], PT[0]-EXTENT/2, PT[0]+EXTENT/2 
LINE_REF = mean(DISP[REFROW-10:REFROW+10,XBEG:XEND], 0) # Reference line profile
tempR = (LINE_REF - LINE_REF.mean())/LINE_REF.std() # Normalize the reference line profile
for j in VERPIX:
    try: 
        tempV = mean(DISP[j-5:j+5,XBEG-MAXSHIFT:XEND+MAXSHIFT], axis=0)
    except: 
        tempV = DISP[j,XBEG-MAXSHIFT:XEND+MAXSHIFT]
    WEIGHTS[j] = sqrt(tempV.mean()**2)
    tempV = (tempV - tempV.mean())/tempV.std() # Normalize the selected line profile
    CORR = correlate(zoom(tempV, UPSAMP), zoom(tempR, UPSAMP), mode='valid')
    SHIFT_HOR[j] = argmax(CORR) # Get the index with maximum correlation
SHIFT_HOR = SHIFT_HOR/UPSAMP - MAXSHIFT
XFIT_YINC = argwhere((abs(SHIFT_HOR)<MAXSHIFT))
YFIT_YINC = SHIFT_VER[XFIT_YINC]
C = polyfit(XFIT_YINC.ravel(), YFIT_YINC.ravel(), 2, w=nan_to_num(WEIGHTS)[XFIT_YINC].ravel()) # Fit a line
SHIFT_HOR_FIT = C[0]*VERPIX**2+C[1]*VERPIX+C[2] # Use the equation to calculate shifts
# XFIT_YINC = argwhere((abs(SHIFT_HOR-SHIFT_HOR_FIT)<3))
# YFIT_YINC = gaussian_filter1d(SHIFT_HOR[XFIT_YINC], 10)
# C = polyfit(XFIT_YINC.ravel(), YFIT_YINC.ravel(), 2, w=nan_to_num(WEIGHTS)[XFIT_YINC].ravel()) # Fit a line
SHIFT_HOR_FIT = C[0]*VERPIX**2+C[1]*VERPIX+C[2] # Use the equation to calculate shifts
SHIFT_HOR_APPLY = -SHIFT_HOR_FIT
previewPlot(XFIT_YINC, YFIT_YINC, 'k-', SHIFT_HOR_FIT, 'k-')

C:\Users\HP-Office\Anaconda2\lib\site-packages\ipykernel\__main__.py:17: RuntimeWarning: invalid value encountered in double_scalars


In [58]:
FLATMEAN_YINCCORR = copy(FLATMEAN_XINCCORR) # Declare shift corrected array
for i in VERPIX: # Calculate the shift corrected array
    shift(FLATMEAN_XINCCORR[i, :], SHIFT_HOR_APPLY[i], FLATMEAN_YINCCORR[i, :], mode='nearest')
FLATMEAN_YINCCORR2 = copy(FLATMEAN_XINCCORR2) # Declare shift corrected array
for i in VERPIX: # Calculate the shift corrected array
    shift(FLATMEAN_XINCCORR2[i, :], SHIFT_HOR_APPLY[i], FLATMEAN_YINCCORR2[i, :], mode='nearest')
previewImage(FLATMEAN_YINCCORR2)

## Line profile removal

In [61]:
ROWS1 = arange(100,400) 
ROWS2 = arange(600,900)
TEMP = append(FLATMEAN_YINCCORR2[ROWS1], FLATMEAN_YINCCORR2[ROWS2], axis=0)
LINE_MED = median(TEMP,0) # Median Line Profile
LINE_MED_NORM = LINE_MED/LINE_MED.max()  # Normalized median line profile
LINE_FILT = gaussian_filter1d(LINE_MED_NORM,2)
FLATMEAN_LINECORR = divide(FLATMEAN_YINCCORR, LINE_FILT) # Declare line profile corrected flat

In [62]:
previewImage(FLATMEAN_LINECORR)
previewPlot(HORPIX, LINE_MED_NORM, HORPIX, LINE_FILT)

### Preview and save

In [63]:
FIG = figure(None, figsize=(14,12))
GRID = GridSpec(4,2)
#-------------------------------------------------------------------
AX1 = FIG.add_subplot(GRID[0:-1,0])
AX1.imshow(FLATMEAN_DARKCORR, cmap='gray', interpolation=None)
AX1.plot(HORPIX, SHIFT_VER_FIT+(YBEG+YEND)/2, 'r', lw=1)
AX1.locator_params(axis='x', tight=True)
AX1.locator_params(axis='y', tight=True)
#-------------------------------------------------------------------
AX2 = FIG.add_subplot(GRID[0:-1,1])
AX2.imshow(FLATMEAN_XINCCORR, cmap='gray', interpolation=None)
AX2.plot(SHIFT_HOR_FIT+(XBEG+XEND)/2, VERPIX, 'r', lw=1)
AX2.locator_params(axis='x', tight=True)
AX2.locator_params(axis='y', tight=True)
#-------------------------------------------------------------------
AX3 = FIG.add_subplot(GRID[-1,0])
AX3.plot(XFIT_XINC,YFIT_XINC, 'k-')
AX3.plot(SHIFT_VER_FIT, 'k-', linewidth=1)
AX3.locator_params(axis='x', tight=True)
AX3.locator_params(axis='y', nbins=5)
AX3.set_xlabel('Column number')
AX3.set_ylabel('Shift \n in pixels')
#-------------------------------------------------------------------
AX4 = FIG.add_subplot(GRID[-1,1])
AX4.plot(XFIT_YINC,YFIT_YINC, 'k-')
AX4.plot(SHIFT_HOR_FIT, 'k-', linewidth=1)
AX4.locator_params(axis='x', tight=True)
AX4.locator_params(axis='y', nbins=5)
AX4.set_xlabel('Row number')
AX4.set_ylabel('Shift \n in pixels')
#-------------------------------------------------------------------
savefig('.\\Plots\\2_Row_col_shifts.png', dpi=600)

In [64]:
FIG = figure(num=None, figsize=(12, 6))
AX1 = FIG.add_subplot(121)
AX2 = FIG.add_subplot(122)
AX1.imshow(FLATMEAN, cmap='gray', interpolation=None)
AX2.imshow(FLATMEAN_LINECORR, cmap='gray', interpolation=None)
tight_layout()
savefig('.\\Plots\\2_Master_flat.png', dpi=600)

## Save properly

In [ ]:
CURRPATHSPLIT = os.path.normpath(FLATFILENAME).split(SEP)  # Components of current file - directory tree
NEWPATH = os.sep.join(CURRPATHSPLIT[0:-3]) + SEP + 'Level-1'  # New directory
DATE = dt.datetime.strftime(dt.datetime.strptime(HEAD['DATE'], '%Y/%m/%d'), '%Y%m%d')
DT = dt.datetime.strptime(HEAD['DATE'] + HEAD['TIME'], '%Y/%m/%d' + '%H:%M:%S')
TZDIFF = dt.timedelta(hours=5, minutes=30)
HEAD['UT'] = dt.datetime.strftime(DT - TZDIFF, '%H:%M:%S')
NEWFILENAME = NEWPATH + SEP + DATE + '_' + CURRPATHSPLIT[-1][0:-5] + 'MASTER.fits'
print('Master flat saved as ' + NEWFILENAME)
NEWTEXTFILENAME = NEWPATH + SEP + DATE + '_' + CURRPATHSPLIT[-1][0:-9] + 'ROWCOL_SHIFTS.txt'
print('X- & Y- inclination corrections saved as ' + NEWTEXTFILENAME)
NEWLINEFILENAME = NEWPATH + SEP + DATE + '_' + CURRPATHSPLIT[-1][0:-9] + 'LINEPROFILE.txt'
print('Line profile  is saved as ' + NEWLINEFILENAME)
#--------------------------------------------------------------------------------------------- Create the text file
ROWS = len(SHIFT_VER_APPLY) + len(SHIFT_HOR_APPLY) + 3
TEXT = array([['-']] * ROWS, dtype='|S100')
TEXT[0, 0] = 'Vertical shifts for each column :'
TEXT[1:len(SHIFT_VER_APPLY) + 1, 0] = SHIFT_VER_APPLY
TEXT[len(SHIFT_VER_APPLY) + 2, 0] = 'Horizonal shifts for each row :'
TEXT[len(SHIFT_VER_APPLY) + 3:len(SHIFT_VER_APPLY) + len(SHIFT_HOR_APPLY) + 3, 0] = SHIFT_HOR_APPLY
TEXT[where(TEXT == '-')] = '\t'
#---------------------------------------------------------------------------------------------------------- Save it
if not os.path.exists(NEWPATH): os.makedirs(NEWPATH)  # Create the new directory if it doesn't exist
HDU_MF = pf.PrimaryHDU(data=uint16(FLATMEAN_LINECORR), header=HEAD)  # Create hdu with data and header
HDU_MF.writeto(NEWFILENAME, overwrite=True)
HDU_IDEAL = pf.PrimaryHDU(data=uint16(ones(FLATMEAN_LINECORR.shape)), header=HEAD)
HDU_IDEAL.writeto(NEWFILENAME[0:-5] + '_IDEAL.fits', overwrite=True)
TEXTFILE = open(NEWTEXTFILENAME, 'w')  # Write things to new file
savetxt(TEXTFILE, TEXT, fmt='{:^20}'.format('%s'), delimiter='\t')
TEXTFILE.close()
savetxt(NEWLINEFILENAME, LINE_MED_NORM, fmt='{:^20}'.format('%f'))
#--------------------------------------------------------------------------------------------------------- Save line profile